In [19]:
from utils import *
from datetime import datetime, timedelta
import requests
import re
from scipy.stats import norm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pyhtml2pdf import  converter
import shutil
import json
from plotly.colors import n_colors

import mimetypes
import os
import smtplib
from email.message import EmailMessage

# First stage

## User defined values

In [3]:
ZentralKurs = 16500
Spannweite = 2000
Schritt = 50
DetailSchritte = 20


# Folders where results will be stored
current_results_path = "./Results"
temp_results_path = "./Results/temp"
old_results_path = "./Results/Old"

# Source file folder
src_path = "./src/"

# Source files
src_html = os.path.join(src_path,"template.html")
src_css = os.path.join(src_path,"style.css")

email_messages_path = os.path.join(src_path,"email_messages")

list_emails_path = os.path.join(src_path,"list_emails_test_Almeida.json")
with open(list_emails_path) as file:
    list_emails = json.load(file)['emails']


list_emails_providers_path = os.path.join(src_path,"email_providers.json")
with open(list_emails_providers_path) as file:
    list_emails_providers = json.load(file)['providers']

list_creditials_path = os.path.join(src_path,"C:\credentials.json")
with open(list_creditials_path) as file:
    list_credentials = json.load(file)['credentials']


# Results files
list_pdf_files = [
    os.path.join(current_results_path, "eurex.pdf"),
    os.path.join(old_results_path, f"eurex_{datetime.today().strftime('%Y_%m_%d')}.pdf")
]
    
list_excel_files = [
    os.path.join(current_results_path, "eurex.xlsx"),
    os.path.join(old_results_path, f"eurex_{datetime.today().strftime('%Y_%m_%d')}.xlsx")
]

result_html = os.path.join(current_results_path,"eurex.html")
result_css = os.path.join(current_results_path,"style.css")
result_image = os.path.join(current_results_path,"image.svg")

# Create results folder in case they still were not created
list_folder_results =  [current_results_path, old_results_path]
for folder in list_folder_results:
    create_folder(folder)

# During the generation of the PDF, the CSS file must be in the same directory
# as the generated html file. So in case the CSS file is still not the in the
# Results folder, we copy this file to it
if not os.path.exists(result_css):
    shutil.copyfile(src_css, result_css)

## Download tradingDates and future_date_col

In [4]:
auswahl = int(input("DAX = 0; STOXX = 1   :"))

if auswahl == 1:
# STOXX
    url = 'https://www.eurex.com/api/v1/overallstatistics/69660'
    print("STOXX")
else:
# DAX
    url = 'https://www.eurex.com/api/v1/overallstatistics/70044'
    print ("DAX")


def get_idx(list_idxs)->list:
    while(True):
        date_idxs = [0, 1]
        #date_idxs = input("Numbers corresponding to the desired dates separated by spaces: ").split(' ')
        date_idxs = list(map(int,date_idxs))
        date_idxs = [idx for idx in date_idxs if idx in list_idxs]
        if len(date_idxs) < 1:
            print("No valid indexes passed.")
        else:
            return date_idxs
    
headers = {'Accept': '*/*'}
params_ov = {'filtertype': 'overview', 'contracttype': 'M'}
print('Getting data from EUREX...')
response = requests.get(url, params=params_ov, headers=headers,timeout = 8)

if not response.ok:
    raise ValueError("Conection failed.")

tradingDates = pd.to_datetime(pd.Series(response.json()['header']['tradingDates']),format="%d-%m-%Y %H:%M")
tradingMeta = response.json()['meta']['isin']

print("Index     Date")
for idx,date in enumerate(tradingDates.dt.strftime("%d-%m-%Y")):
    print(f" {idx:<6} {date:^3}")

date_idxs = get_idx(tradingDates.index)
params_ov['busdate'] = tradingDates[0].strftime("%Y%m%d")
response = requests.get(url, params=params_ov, headers=headers,timeout = 8)
if not response.ok:
        raise ValueError("Conection failed.")

params_details = {}
if 'dataRows' in response.json(): 
    overview_df = pd.DataFrame(response.json()['dataRows'])
    overview_df = overview_df[overview_df.contractType == 'M']
    overview_df.sort_values('date',ignore_index=True,inplace=True)
    future_date_col = overview_df.loc[:,'date']
    overview_df.date = pd.to_datetime(overview_df.date,format='%Y%m%d').dt.strftime('%d-%m-%Y')
    display(overview_df)
else:
    raise ValueError

params_details['busdate'] = f"{tradingDates[1].strftime('%Y%m%d')}"
response = requests.get(url, params=params_details, headers=headers,timeout = 8)
if not response.ok:
    raise ValueError("Conection failed.")

DAX
Getting data from EUREX...
Index     Date
 0      01-09-2023
 1      31-08-2023
 2      30-08-2023
 3      29-08-2023
 4      28-08-2023
 5      25-08-2023
 6      24-08-2023
 7      23-08-2023
 8      22-08-2023
 9      21-08-2023
 10     18-08-2023
 11     17-08-2023
 12     16-08-2023
 13     15-08-2023
 14     14-08-2023
 15     11-08-2023
 16     10-08-2023
 17     09-08-2023
 18     08-08-2023
 19     07-08-2023
 20     04-08-2023


,date,callVolume,callOpenInterest,putVolume,putOpenInterest,putCallRatio,total,contractType
0,15-09-2023,16322.0,151245.0,33157.0,332210.0,2.031,49479.0,M
1,20-10-2023,3524.0,25024.0,21834.0,58195.0,6.196,25358.0,M
2,17-11-2023,1094.0,4329.0,921.0,7538.0,0.842,2015.0,M
3,15-12-2023,1235.0,162605.0,10141.0,400174.0,8.211,11376.0,M
4,15-03-2024,442.0,27015.0,7822.0,114806.0,17.697,8264.0,M
5,21-06-2024,1697.0,29916.0,556.0,63228.0,0.328,2253.0,M
6,20-09-2024,1.0,1113.0,495.0,7084.0,495.000,496.0,M
7,20-12-2024,0.0,7371.0,18.0,13029.0,n/a,18.0,M
8,21-03-2025,0.0,1.0,0.0,5.0,n/a,0.0,M
9,20-06-2025,0.0,7001.0,0.0,2.0,n/a,0.0,M


**tbd: heute shall be a working day (Mo - Fr)**

In [5]:
expiry = datetime.strptime(future_date_col[0],"%Y%m%d")
expiry_1 = datetime.strptime(future_date_col[1],"%Y%m%d")


heute = datetime.today()
if heute.weekday() > 4:
    #sunday => wd = 6 -> td = +1
    #sartuday => wd = 5 > td +2
    heute += timedelta(days= 7 - heute.weekday())
heute += timedelta(days=0)
print(f'heute = {heute.strftime("%d/%m/%Y")}')

tage_bis_verfall = (expiry - heute).days +1

df_StgrtDAX = pd.read_html("https://www.boerse-stuttgart.de/en/",match="L&S DAX")[0]
display(df_StgrtDAX)


# Cntralize diagram
DAX_LS_e = df_StgrtDAX.loc[df_StgrtDAX['Indices GER'] == "L&S DAX","Price"].values[0] #X4

print(f"ZentralKurs value was changed: {ZentralKurs} => ",end="")
if (ZentralKurs - DAX_LS_e) > 500:
    ZentralKurs -= 500
    ZentralKurs_has_changed = True
elif (ZentralKurs - DAX_LS_e) < -500:
    ZentralKurs -= 500
    ZentralKurs_has_changed = True
print(f"{ZentralKurs}")

heute = 01/09/2023


,Indices GER,Price,%,Time
0,Citi DAX,15856.62,-0.56,9:59 PM
1,L&S DAX,15862.00,-0.52,10:05 PM
2,DAX®,15840.34,-0.67,5:55 PM
3,MDAX,27812.93,-0.02,5:55 PM
4,TecDAX,3158.79,-0.58,5:55 PM
5,SDAX,13324.21,-0.56,5:55 PM
6,EUWAX Sentiment,26.90,-,8:00 PM


ZentralKurs value was changed: 16500 => 16000


In [6]:
Minkurs = ZentralKurs - (Spannweite/ 2)
Maxkurs = Minkurs + Spannweite
Schritte = Spannweite / Schritt

DetailMin = round(DAX_LS_e - (Spannweite)/4)
DetailMax = DetailMin + (Spannweite/2)


if tage_bis_verfall >= 1:
    delta = 0.5
else:
    delta = 1

is_delta_checked = False

# Second stage

In [7]:
# Create Basis column that will be used for multiple tables
Basis = pd.Series((Minkurs + np.arange(int(Schritte) +1)* Schritt)[::-1])

# Transform series into dataframe to make dataframe creation easier 
Basis_df = Basis.reset_index().rename(columns={0:"Basis"})[["Basis"]]


# Ueberhaenge_df, Summery_df are created from a copy of the same Dataframe
# so they have the same index. It means they could be in one  Dataframe but
#  I decided to keep them separated as they were in the VBA code.

Ueberhaenge_df = Basis_df.copy()
Summery_df = Basis_df.copy()

params_details = {}
params_details['filtertype'] = 'detail'
params_details['contracttype'] = 'M'

for productdate_idx in [0,1]:
    params_details['productdate'] = future_date_col[productdate_idx]
    for busdate_idx in [0,1]:
        params_details['busdate'] = f"{tradingDates[busdate_idx].strftime('%Y%m%d')}" 
        print(f"busdate_idx = {busdate_idx}   | productdate_idx = {productdate_idx}")
        
        if (busdate_idx == 1) and (productdate_idx == 1):
            # No need to request data for this case so we skip this iteration
            continue
        
        # Request data
        response = requests.get(url, params=params_details, headers=headers, timeout = 10)
        if not response.ok:
            raise ValueError("Conection failed.")
        contractsCall_aux_df =  pd.DataFrame(response.json()['dataRowsCall'])
        contractsPut_aux_df =  pd.DataFrame(response.json()['dataRowsPut'])

        # Create dataframe with the just requested data
        aux_df = Basis_df.copy()

        aux_df = aux_df.merge(
            contractsCall_aux_df[['strike','openInterest']],
            left_on = "Basis",
            right_on = "strike",
            how= "left"
        )

        aux_df = aux_df.merge(
            contractsPut_aux_df[['strike','openInterest']],
            on = "strike",
            how="left",
            suffixes=('_CF', '_PF')
        )

        if productdate_idx == 0:
            Ueberhaenge_df[busdate_idx] = aux_df.openInterest_PF - aux_df.openInterest_CF

            if busdate_idx == 0:
                Summery_df["openInterest_PF"] = aux_df["openInterest_PF"]
                Summery_df["openInterest_CF"] = aux_df["openInterest_CF"]
        
        elif productdate_idx == 1:
            if busdate_idx == 0:
                Ueberhaenge_df["nextContract"] = aux_df.openInterest_PF - aux_df.openInterest_CF
                Summery_df["nextContract"] = Ueberhaenge_df["nextContract"]


Ueberhaenge_df.rename(columns = {0 : "Front"},inplace=True)
Summery_df["heute"] = Ueberhaenge_df["Front"] * 0.2* delta

Ueberhaenge_df.rename(columns = {1 : "Last"},inplace=True)
Summery_df["last_day"] = Ueberhaenge_df["Last"] * 0.2* delta
display(Ueberhaenge_df.head())
display(Summery_df.head())

busdate_idx = 0   | productdate_idx = 0
busdate_idx = 1   | productdate_idx = 0
busdate_idx = 0   | productdate_idx = 1
busdate_idx = 1   | productdate_idx = 1


,Basis,Front,Last,nextContract
0,17000.0,-4020.0,-6221.0,-1123.0
1,16950.0,-511.0,-511.0,-235.0
2,16900.0,-4302.0,-4302.0,-1165.0
3,16850.0,-649.0,-612.0,-63.0
4,16800.0,-3775.0,-3817.0,-2692.0


,Basis,openInterest_PF,openInterest_CF,nextContract,heute,last_day
0,17000.0,1163.0,5183.0,-1123.0,-402.0,-622.1
1,16950.0,0.0,511.0,-235.0,-51.1,-51.1
2,16900.0,204.0,4506.0,-1165.0,-430.2,-430.2
3,16850.0,41.0,690.0,-63.0,-64.9,-61.2
4,16800.0,226.0,4001.0,-2692.0,-377.5,-381.7


In [8]:
Ueberhaenge_df["Summe"] = Ueberhaenge_df[["Front", "nextContract"]].sum(axis=1)
Ueberhaenge_df = Ueberhaenge_df[['Basis', 'Summe',"Last",'Front', "nextContract"]]
display(Ueberhaenge_df.head())
display(Summery_df.head())

,Basis,Summe,Last,Front,nextContract
0,17000.0,-5143.0,-6221.0,-4020.0,-1123.0
1,16950.0,-746.0,-511.0,-511.0,-235.0
2,16900.0,-5467.0,-4302.0,-4302.0,-1165.0
3,16850.0,-712.0,-612.0,-649.0,-63.0
4,16800.0,-6467.0,-3817.0,-3775.0,-2692.0


,Basis,openInterest_PF,openInterest_CF,nextContract,heute,last_day
0,17000.0,1163.0,5183.0,-1123.0,-402.0,-622.1
1,16950.0,0.0,511.0,-235.0,-51.1,-51.1
2,16900.0,204.0,4506.0,-1165.0,-430.2,-430.2
3,16850.0,41.0,690.0,-63.0,-64.9,-61.2
4,16800.0,226.0,4001.0,-2692.0,-377.5,-381.7


In [9]:
Summery_df["Änderung"] = Summery_df.heute - Summery_df.last_day

if (delta == 1) and (not is_delta_checked):
    Summery_df['nextContract'] =  Summery_df['nextContract'] / 2
    is_delta_checked= True
    print(f"is_delta_checked = {is_delta_checked}")
Summery_df.head()

,Basis,openInterest_PF,openInterest_CF,nextContract,heute,last_day,Änderung
0,17000.0,1163.0,5183.0,-1123.0,-402.0,-622.1,220.1
1,16950.0,0.0,511.0,-235.0,-51.1,-51.1,0.0
2,16900.0,204.0,4506.0,-1165.0,-430.2,-430.2,0.0
3,16850.0,41.0,690.0,-63.0,-64.9,-61.2,-3.7
4,16800.0,226.0,4001.0,-2692.0,-377.5,-381.7,4.2


In [10]:
SummeryDetail_df = Summery_df[(Summery_df.Basis >= DetailMin) & (Summery_df.Basis < (DetailMax + Schritt))]
SummeryDetail_df.head()

,Basis,openInterest_PF,openInterest_CF,nextContract,heute,last_day,Änderung
12,16400.0,844.0,3710.0,-1112.0,-286.6,-284.2,-2.4
13,16350.0,89.0,1327.0,-292.0,-123.8,-122.5,-1.3
14,16300.0,3482.0,5798.0,-1034.0,-231.6,-441.6,210.0
15,16250.0,1920.0,1395.0,-798.0,52.5,59.8,-7.3
16,16200.0,3175.0,4938.0,-357.0,-176.3,-168.2,-8.1


# Third stage

In [11]:
euribor_3m_df = pd.read_html("https://www.euribor-rates.eu/en/current-euribor-rates/2/euribor-rate-3-months/")[0].rename(columns= {0: "Date", 1 : "InterestRate"})
euribor_row = euribor_3m_df.loc[0]
print(f"Euribor 3M in {euribor_row['Date']} is {euribor_row['InterestRate']}")
euribor_3m_df.head()

Euribor 3M in 8/31/2023 is 3.795 %


,Date,InterestRate
0,8/31/2023,3.795 %
1,8/30/2023,3.803 %
2,8/29/2023,3.783 %
3,8/28/2023,3.771 %
4,8/25/2023,3.788 %


In [12]:
response_txt = requests.get("https://qontigo.com/index/vdax2m/").text
data = re.findall(r"window.chart_data_two_week = (.*);", response_txt)[0]
data = eval(f"np.array({data},float)")[-10:][::-1]
vda_2m_df = pd.DataFrame(np.asarray(data[:,0], dtype='datetime64[ms]'),columns=["Date"])
vda_2m_df["Price"] = data[:,1]
vda_2m_row = vda_2m_df.loc[0]
print(f"VDA-NEW 2M in {vda_2m_row['Date']} is € {vda_2m_row['Price']}")
vda_2m_df

VDA-NEW 2M in 2023-08-30 17:30:00 is € 15.1939


,Date,Price
0,2023-08-30 17:30:00,15.1939
1,2023-08-30 17:15:00,15.2227
2,2023-08-30 17:00:00,15.2592
3,2023-08-30 16:45:00,15.2621
4,2023-08-30 16:30:00,15.3469
5,2023-08-30 16:15:00,15.1754
6,2023-08-30 16:00:00,15.1756
7,2023-08-30 15:45:00,15.2727
8,2023-08-30 15:30:00,15.2895
9,2023-08-30 15:15:00,15.2460


In [13]:
SchrittWeite = 10
InterestRate = float(euribor_row["InterestRate"].replace("%",""))/100
VDAX = vda_2m_row["Price"]/100
VDAX_Laufzeit = 60

Tage = tage_bis_verfall
if Tage == 0:
    Tage = 0.5

Tage_1 = (expiry_1 - heute).days +1

In [14]:
Kurs_count = int(Spannweite/SchrittWeite)+1 
HedgeBedarf_kurs=  pd.DataFrame(Maxkurs - np.arange(Kurs_count)* SchrittWeite,columns= ["Basis"])


Hedge_dimensions = Kurs_count,int(Schritte+1)
HedgeBedarf_values = np.zeros(Hedge_dimensions)
HedgeBedarf1_values = np.zeros(Hedge_dimensions)


for k in range(Hedge_dimensions[1]):
    Basis_value = Basis[k]
    Kontrakte = Ueberhaenge_df.loc[k,"Front"]
    Kontrakte_1 = Ueberhaenge_df.loc[k,"nextContract"]
    for i in range(Hedge_dimensions[0]):
        Kurs = Maxkurs - i * SchrittWeite
        # In python np.log = natural log
        h1 = np.log(Kurs / Basis_value)
        sigma = VDAX * ((Tage / VDAX_Laufzeit) ** 0.5)
        sigma_1 = VDAX * ((Tage_1 / VDAX_Laufzeit) ** 0.5)
        h2 = InterestRate + sigma * sigma / 2
        h2_1 = InterestRate + sigma_1 * sigma_1 / 2
        d1 = (h1 + (h2 * (Tage / 365))) / (sigma * ((Tage / 365) ** 0.5))

        d1_1 = (h1 + (h2_1 * (Tage_1 / 365))) / (sigma_1 * ((Tage_1 / 365) ** 0.5))

        Phi = norm.pdf(d1, 0, 1)               
        Phi_1 = norm.pdf(d1_1, 0, 1)

        Gamma = Phi / (Kurs * (sigma * (Tage / 365) ** 0.5))
        Gamma_1 = Phi_1 / (Kurs * (sigma_1 * (Tage_1 / 365) ** 0.5))
        HedgeBedarf_values[i,k] = Gamma * Kontrakte / 5
        HedgeBedarf1_values[i,k] = Gamma_1 * Kontrakte_1 / 5


HedgeSum = HedgeBedarf_values.sum(axis=1)/2
HedgeSum_1 = HedgeBedarf1_values.sum(axis=1)/2

In [15]:
HedgeBedarf_df = pd.DataFrame(data =HedgeBedarf_values, columns=Basis)
HedgeSum_df = pd.DataFrame(HedgeSum,columns=["HedgeSum"])
HedgeBedarf_df = pd.concat([HedgeBedarf_kurs,HedgeSum_df,HedgeBedarf_df], axis=1)

HedgeBedarf1_df = pd.DataFrame(data =HedgeBedarf1_values, columns=Basis)
HedgeSum1_df = pd.DataFrame(HedgeSum_1,columns=["HedgeSum"])
HedgeBedarf1_df = pd.concat([HedgeBedarf_kurs,HedgeSum1_df,HedgeBedarf1_df], axis=1)

# Fourth Stage

In [16]:
auswahl_dict = {
    0 : "DAX",
    1 : "STOXX"
}

info_df = pd.DataFrame({
    'ZentralKurs' : [ZentralKurs],
    'Spannweite': [Spannweite],
    'SchrittWeite': [SchrittWeite],
    'Schritt' : [Schritt],
    'auswahl': auswahl_dict[auswahl],
    'expiry': expiry,
    'expiry_1': expiry_1,
    'heute': heute,
    #"nextContract": expiry_1,
    'tage_bis_verfall': tage_bis_verfall,
    'DAX_LS_e': DAX_LS_e,
    'Minkurs': Minkurs,
    'Maxkurs': Maxkurs,
    'Schritte': Schritte,
    'DetailMin': DetailMin,
    'DetailMax': DetailMax,
    'delta' : delta,
    'Euribor_IR': euribor_row['InterestRate'],
    'VDA_price': vda_2m_row['Price'],
}).T.reset_index().rename(columns={0:"Value", 'index': "Info"})
info_df

,Info,Value
0,ZentralKurs,16000
1,Spannweite,2000
2,SchrittWeite,10
3,Schritt,50
4,auswahl,DAX
5,expiry,2023-09-15 00:00:00
6,expiry_1,2023-10-20 00:00:00
7,heute,2023-09-01 21:25:47.626413
8,tage_bis_verfall,14
9,DAX_LS_e,15862.0


In [17]:
for excel_file in list_excel_files:
    with pd.ExcelWriter(excel_file,datetime_format="DD/MM/YYYY") as writer:
        info_df.to_excel(writer,sheet_name='infos',index=False)
        Ueberhaenge_df.to_excel(writer,sheet_name='Ueberhaenge',index=False)
        Summery_df.to_excel(writer,sheet_name='Summery',index=False)
        SummeryDetail_df.to_excel(writer,sheet_name='SummeryDetail',index=False)
        overview_df.to_excel(writer,sheet_name='Overview',index=False)
        HedgeBedarf_df.to_excel(writer,sheet_name='HedgeBedarf',index=False)
        HedgeBedarf1_df.to_excel(writer,sheet_name='HedgeBedarf+01',index=False)
        vda_2m_df.to_excel(writer,sheet_name='VDAX-New 2M Hist',index=False)
        euribor_3m_df.to_excel(writer,sheet_name='EURIBOR 3M',index=False)
        
print("Files have been exported.")
HedgeBedarf_df["Sum"] = HedgeBedarf_df.HedgeSum + HedgeBedarf1_df.HedgeSum

Files have been exported.


# Fifth stage

In [20]:
# Values to create to arrow
idx_closest = (HedgeBedarf_df.Basis - DAX_LS_e).abs().idxmin()
closest_Basis =  HedgeBedarf_df.loc[idx_closest,"Basis"]
closest_Sum = HedgeBedarf_df.loc[idx_closest,"Sum"]

x_axis_length = HedgeBedarf_df.Sum.max() - HedgeBedarf_df.Sum.min()
y_axis_length = HedgeBedarf_df.Basis.max() - HedgeBedarf_df.Basis.min()



min_Kontrakte = 5000
max_Differenz = 1000
prozentual = 0.2

# (CF > min_Kontrakte) AND (PF > min_Kontrakte) AND (ABS(CF - PF)  < MIN(PF,CF)*)
mask_highlights = (
    (Summery_df.openInterest_CF > min_Kontrakte) & 
    (Summery_df.openInterest_PF > min_Kontrakte) & 
    (
        (Summery_df.openInterest_CF - Summery_df.openInterest_PF).abs() < 
        (Summery_df[['openInterest_PF',"openInterest_CF"]].min(axis=1)*prozentual)))

################################## Hilights for Basis column ####################################

Summery_df["basis_color"] = "lavender"
Summery_df.loc[
    mask_highlights, "basis_color"] = "yellow"
col_basis_color = Summery_df.basis_color.to_numpy()
#################################################################################################

######################### Gradient of red and blue for Anderung column ##########################
aux = Summery_df.Änderung.reset_index()
aux['r'] = aux['g']  = aux['b'] = 255
aux.loc[aux.Änderung<0,'g'] = aux.loc[aux.Änderung<0,'b'] = 255 - 150*aux.Änderung/(aux.Änderung.min()) 
aux.loc[aux.Änderung>0,'g'] = aux.loc[aux.Änderung>0,'r'] = 255 - 150*aux.Änderung/(aux.Änderung.max())
aux = aux.astype(float)
rb_shades = np.array([f"rgb({aux.loc[i,'r']},{aux.loc[i,'g']},{aux.loc[i,'b']})" for i in range(aux.shape[0]) ])
col_anderung_color = rb_shades
#################################################################################################

############################# Colors for heute and last_day columns #############################
col_heute_color = posneg_binary_color(Summery_df.heute,"rgb(0, 204, 204)","rgb(77, 166, 255)")
col_last_day_color = posneg_binary_color(Summery_df.last_day,"rgb(0, 204, 204)","rgb(77, 166, 255)")
#################################################################################################

###################### Gradient of green and blue for Put anc Call columns ######################
col_put_color = colors = np.array(n_colors('rgb(214, 245, 214)', 'rgb(40, 164, 40)',
    20, colortype='rgb'))[scale_col_range(Summery_df.openInterest_PF,19)]
col_call_color = colors = np.array(n_colors('rgb(204, 224, 255)', 'rgb(0, 90, 179)',
    20, colortype='rgb'))[scale_col_range(Summery_df.openInterest_CF,19)]
#################################################################################################

row_height = 25
# 41 row + header
height = row_height*42

# width of the image
widht = 1000

# Create figure containg two elements
fig = make_subplots(
    # 2 Columns which are table and chart
    rows=1, cols=2,
    # Width ocupied by table and chart
    column_widths=[13,7],

    # Define types of the figures and margins
    specs=[[
        {"type": "table"},

        {"type": "scatter",
        # Add margins to the top and to the bottom so Basis column match the y axis of the chart 
        "t" : 1.5*(row_height/height), 'b':0.75*(row_height/height)
        }, 
        ]],
           )

# Create table figure
fig.add_trace(
    # Choose element 1 => table
    row = 1, col = 1,
    trace = go.Table(

    # Define some paremeters for the header
    header=dict(
        # Names of the columns
        values=(bold(["Basis","Änderung",heute.strftime("%d/%m/%Y"),tradingDates[1].strftime("%d/%m/%Y"),"Put", "Call"])),
        
        # Header style
        fill_color='paleturquoise',
            align='center',
            font = {'size': 11},
            height = row_height
        ),

    cells=dict(

        align='center',
        height = row_height,
        font = {'size': 11,},

        # Values of the table
        values=[
            (Summery_df.Basis),
            (Summery_df.Änderung.round(2)),
            (Summery_df.heute.round(2)),
            (Summery_df.last_day.round(2)),
            (Summery_df.openInterest_PF), 
            (Summery_df.openInterest_CF)
        ],

        # Colors of the columns
        fill_color = [
            col_basis_color,
            col_anderung_color,
            col_heute_color,
            col_last_day_color,
            col_put_color, 
            col_call_color
        ],
    )
))

##################################### Add plots in the chart #####################################
fig.add_trace(go.Scatter(
    x = HedgeBedarf_df.Sum,
    y = HedgeBedarf_df.Basis,
    mode = "lines",
    name = heute.strftime("%Y-%m") +" + " + expiry.strftime("%Y-%m"),
    marker_color= "blue"
    ),
)

fig.add_trace(go.Scatter(
    x = HedgeBedarf1_df.HedgeSum,
    y = HedgeBedarf_df.Basis,
    mode = "lines",
    name = expiry.strftime("%Y-%m"),
    marker_color= "purple"
    ),
)

fig.add_trace(go.Scatter(
    x = HedgeBedarf_df.HedgeSum,
    y = HedgeBedarf_df.Basis,
    mode = "lines",
    name = heute.strftime("%Y-%m"),
    marker_color= "orange"
))

# Create arrow
fig.add_trace(go.Scatter(
        x=[closest_Sum + x_axis_length/5,closest_Sum + x_axis_length/50], 
    y=[closest_Basis,closest_Basis],
    marker= dict(size=20,symbol= "arrow-bar-up", color="red", angleref="previous"),
    showlegend = False
    )
)
##################################################################################################
dx =  0.1*(HedgeBedarf_df.Sum.max() - HedgeBedarf_df.Sum.min())

fig.update_layout(
    # Set limits in the x and y axis
    yaxis_range= [HedgeBedarf_df.Basis.min(), HedgeBedarf_df.Basis.max()],
    xaxis_range= [HedgeBedarf_df.Sum.min() - dx, HedgeBedarf_df.Sum.max() + dx],
    
    # Remove margins
    margin=dict(l=0,r=0,b=0,t=0),
    paper_bgcolor="white",
    
    # Define width and height of the image
    width=widht,height=height + row_height,
    template = "seaborn",

    # Legend parameters
    legend=dict(
        yanchor="top",
        y=0.98 - (row_height/height),
        xanchor="right",
        x= 0.98,
        font=dict(
            size = 10
        ),
        # legend in the vertical
        orientation = "v"
        )
)

fig.show()

fig.write_image(result_image,scale=1)

with open(src_html) as file:
    template = file.read()

# Replace specific characters in the template by values
dict_raplace = {
    "$PUT_SUM$": int(Summery_df.openInterest_PF.sum()),
    "$CALL_SUM$": int(Summery_df.openInterest_CF.sum()),
    "$TBF$": tage_bis_verfall,
    "$DELTA$":str(delta).replace(".",","),
    "$DATE$": heute.strftime("%d/%m/%Y")
}

for key, value in dict_raplace.items():
    template = template.replace(key, str(value))

# Export html file
with open(result_html,'w') as file:
    file.write(template)

# Export pdf files
for pdf_file in list_pdf_files:
    converter.convert("file://" + os.path.join(os.getcwd(),result_html), pdf_file)
print("PDF has been generated.")

PDF has been generated.


# Sixth stage

In [23]:
def send_mail(send_from: str, send_to: list, subject: str, body: str, filepath : str, smtp_url, port, username, password):

    # Create the email message
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = send_from
    msg['To'] = ', '.join(send_to)
    # Set email content
    msg.set_content(message)

    path = directory + filename

    if os.path.exists(path):
        ctype, encoding = mimetypes.guess_type(path)
        if ctype is None or encoding is not None:
            # No guess could be made, or the file is encoded (compressed), so
            # use a generic bag-of-bits type.
            ctype = 'application/octet-stream'
        maintype, subtype = ctype.split('/', 1)
        # Add email attachment
        with open(path, 'rb') as fp:
            msg.add_attachment(fp.read(),
                           maintype=maintype,
                           subtype=subtype,
                           filename=filename)

    smtp = smtplib.SMTP(smtp_url, port)
    smtp.starttls()
    smtp.login(username, password)
    smtp.send_message(msg)
    smtp.quit()

In [24]:
msg = EmailMessage()

In [20]:
list_pdf_files

['./Results\\eurex.pdf', './Results/Old\\eurex_2023_09_01.pdf']

In [ ]:
for email in list_emails:
    credentials = list_credentials[email['from']]
    smtp_config = list_emails_providers[credentials['provider']]['smtp']
    if list_conditions[email['condition']]:
        send_mail(
            send_from = email['from'],
            send_to = email['mailto'],
            subject = email['subject'],
            body = open(os.path.join(email_messages_path,email['text']),encoding="UTF-8").read(),
            directory = './',
            filename = 'requirements.txt',
            smtp_url = smtp_config['server'],
            port = smtp_config['port'],
            username = email['from'], 
            password = credentials['password']
        )

almeidabarrosdf@gmail.com
dummy_eurex@outlook.com


In [ ]:
# construct Outlook application instance
outlook = client.Dispatch('Outlook.Application')

if check_emails_available_outlook(outlook, [email['from'] for email in list_emails]):
    print("All e-mails to send from are available on the outlook session.")

    
for email in list_emails:
    if list_conditions[email['condition']]:
        account = outlook.Session.Accounts[email['from']]
        # construct the email item object
        message = outlook.CreateItem(0)
        message.Subject = email['subject']
        message.Body = open(os.path.join(email_messages_path,email['text']),encoding="UTF-8").read()
        message.To = ";".join(email['mailto'])
        message._oleobj_.Invoke(*(64209, 0, 8, 0, account))
        message.Attachments.Add(os.path.join(os.getcwd(), 'requirements.txt'))
        #message.Display()
        message.Save()
        message.Send()